Use Python to fetch XAUUSDc H1 candles data from MT5.  Add a signal logic: Starts calculating exactly at 0:00am UTC, If the corresponding candle's tick-volume bar 'position =0' is lower than 'position = -1' and  "'position = -1' is higher than 'position = -2'  and 'position = 0' ", then highlight the coresponding candle red of bulish and blue if bullish. Use plotly white themed.

import MetaTrader5 as mt5
import pandas as pd
import plotly.graph_objects as go
from datetime import datetime, timezone

# Initialize MT5
if not mt5.initialize():
    print("MT5 initialization failed")
    mt5.shutdown()

# Fetch H1 candles for XAUUSDc
symbol = "XAUUSDc"
timeframe = mt5.TIMEFRAME_H1
utc_now = datetime.now(timezone.utc)
rates = mt5.copy_rates_from(symbol, timeframe, utc_now, 2000)
mt5.shutdown()

# Convert to DataFrame
df = pd.DataFrame(rates)
df['time'] = pd.to_datetime(df['time'], unit='s', utc=True)

# Calculate tick-volume signal
df['signal'] = ''
for i in range(2, len(df)):
    pos0 = df.loc[i, 'tick_volume']
    pos1 = df.loc[i-1, 'tick_volume']
    pos2 = df.loc[i-2, 'tick_volume']
    
    # Only at 0:00 UTC
    if df.loc[i, 'time'].hour == 0:
        if pos0 < pos1 and pos1 > max(pos2, pos0):
            df.loc[i, 'signal'] = 'bullish' if df.loc[i, 'close'] > df.loc[i, 'open'] else 'bearish'

# Colors for highlighted candles
colors = ['blue' if s=='bullish' else 'red' if s=='bearish' else 'black' for s in df['signal']]

# Create figure with secondary y-axis (row 2 for volume)
fig = go.Figure()

# Candlestick trace
fig.add_trace(go.Candlestick(
    x=df['time'],
    open=df['open'],
    high=df['high'],
    low=df['low'],
    close=df['close'],
    increasing_line_color='black',
    decreasing_line_color='black',
    name='Price'
))

# Overlay colored signal candles
for i, row in df.iterrows():
    if row['signal'] in ['bullish', 'bearish']:
        fig.add_trace(go.Candlestick(
            x=[row['time']],
            open=[row['open']],
            high=[row['high']],
            low=[row['low']],
            close=[row['close']],
            increasing_line_color=colors[i],
            decreasing_line_color=colors[i],
            showlegend=False
        ))

# Tick-volume bar chart in separate pane
fig.add_trace(go.Bar(
    x=df['time'],
    y=df['tick_volume'],
    marker_color='grey',
    name='Tick Volume',
    yaxis='y2'
))

# Layout update
fig.update_layout(
    template='plotly_white',
    title=f"{symbol} H1 Candle Signals",
    xaxis_title="Time",
    yaxis_title="Price",
    yaxis2=dict(
        title='Tick Volume',
        overlaying='y',
        side='right',
        showgrid=False,
        position=0.15
    ),
    xaxis=dict(rangeslider=dict(visible=False)),
    height=800,  # Make chart bigger
)

fig.show()
